# Exploring Database Tables

In [ ]:
-- Execute this cell to return the list of tables
-- and views within the Fossils schema of this database
-- (Use \help to learn more options)

\list Fossils.*

In [ ]:
-- Execute this cell to return the list of tables
-- and views within the default schema of this database
-- (Use \help to learn more options)

\list

In [ ]:
-- Explore RDD_Country_Budget table
SELECT *
FROM Country_Budgets_Combined_Details
-- GROUP BY COUNTRY;
LIMIT 100;

In [ ]:
-- Explore RDD_Country_Budget table by PRODUCT
SELECT PRODUCT
FROM Country_Budgets_Combined_Details
GROUP BY PRODUCT
LIMIT 100;

In [ ]:
-- Explore RDD_Country_Budget table by PRODUCT
SELECT PRODUCT
FROM PERGDP
GROUP BY PRODUCT
LIMIT 100;